# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from A3C_Agent import *
from ENV_DETAILS import *

2023-03-27 07:26:31.527753: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\Acrobot\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/Acrobot/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_ddqn/' # Linux
    
ENV = "Acrobot-v1"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env


<TimeLimit<OrderEnforcing<PassiveEnvChecker<AcrobotEnv<Acrobot-v1>>>>>

In [4]:
s = env.observation_space.sample()
s

array([-0.7952974 , -0.18022607, -0.6180601 , -0.4638386 ,  3.5308254 ,
       14.131643  ], dtype=float32)

In [5]:
env.observation_space, env.observation_space.shape

(Box([ -1.        -1.        -1.        -1.       -12.566371 -28.274334], [ 1.        1.        1.        1.       12.566371 28.274334], (6,), float32),
 (6,))

In [6]:
s = env.reset()
s[0]

array([ 9.9932265e-01,  3.6799867e-02,  9.9663723e-01, -8.1940420e-02,
        2.9496226e-02, -7.4400299e-04], dtype=float32)

In [7]:
s = env.reset()[0]
env.action_space.n, env.action_space, env.step(1)

(3,
 Discrete(3),
 (array([ 0.99623036,  0.08674699,  0.99932665,  0.03669089, -0.00785548,
          0.08642782], dtype=float32),
  -1.0,
  False,
  False,
  {}))

# Advantage Asynch Actor-Critic 

This time we will implement the A3C not considering Actor and Critic as part of the same network

### Training/Hyperparam run

In [8]:
%tensorboard --logdir logs_general/hyper

UsageError: Line magic function `%tensorboard` not found.


In [9]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_A3C/"

if TUNING_TYPE == "MANUAL":
    params = {}
    params["n_enviroment"] =[20]
    params["discount"] =[0.96, 0.97]
    params["end_of_episode"] = [400]
    params["learning_rate"] = [0.001]
    params["entropy_factor"] = [ 0.01, 0.05]
    params["epsilon"] = [1]
    params["boltzman_factor"] = [1]
    params["exploration_technique"] = ['soft', 'epsilon']
    params["training_steps"] = [3000000]
    params["dense_units"] = [32, 128]
    params["time_to_update"] = [400]
    params["use_LSTM"] =[False]


    hyperparam_combination = list(itertools.product(*list(params.values())))
    

    try:

        files = [name for name in os.listdir(logs_dir) if os.path.isfile(os.path.join(logs_dir, name)) and name != "logfile.txt" and name != "merged_results.json"]
        if len(files) >= 1: merge_JsonFiles(main_hyper_dir, logs_dir, files)

        res_file = logs_dir+"merged_results.json"
        def without_keys(d, keys):
            return {x: d[x] for x in d if x not in keys}


        if os.path.isfile(res_file):
            with open(res_file, 'r') as f:
                complete_file = json.load(f)

            newlist = sorted(complete_file, key=lambda d: d['mean_rewards'], reverse=True) 
            params = []
            for i, f in enumerate(newlist):
                label = "disc : "+str(f['discount'])+" | "+"lr : "+str(f['learning_rate'])+" | "+"entropy : "+str(f['entropy_factor'])+" | "+"update : "+str(f['time_to_update'])
                plt.figure(figsize=[35,4])
                plt.plot(f['rewards'], label = label)
                plt.legend()
                max_mean_reward = f['mean_rewards']
                params.append(f)
                print(without_keys(f,"rewards"))
                if i == 10:
                    break
            plt.title("Evaluation rewards"); plt.grid();
            plt.show()

    except:
        pass



In [10]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_A3C"
    project_name = "keras_tunning_soft"

    tuner = kt.BayesianOptimization(
            MyHyperModel(hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/A3C_soft/", 
                          end_of_episode = EPISODES, n_enviroment = 5, 
                  evaluation_epoch = 2000, training_steps = 600000,
                  sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                  discount_min = 0.90, discount_max = 0.99,
                  entropy_min = 0.0001, entropy_max = 0.05,
                  lr_min = 0.00001, lr_max = 0.005,
                  dense_min = 32, dense_max = 500,
                  lstm_min = 32, lstm_max = 128,
                  time_to_update_min = 100, time_to_update_max=600,
                  environment_name=ENV),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 50,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
    
    # project_name = "keras_tunning_epsilon"
    # tuner = kt.BayesianOptimization(
    #         MyHyperModel(hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/A3C_epsilon/", exploration_tech = "epsilon"),
    #         objective= kt.Objective('total_train_reward', direction="max"), 
    #         max_trials = 50,
    #         # distribution_strategy= strategy,
    #         directory=dir,
    #         project_name=project_name
    #     )
    # tuner.search(x=[0], y=[1])

    # project_name = "keras_tunning_boltzman"
    # tuner = kt.BayesianOptimization(
    #         MyHyperModel(hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/A3C_boltzman/", exploration_tech = "boltzman"),
    #         objective= kt.Objective('total_train_reward', direction="max"), 
    #         max_trials = 50,
    #         # distribution_strategy= strategy,
    #         directory=dir,
    #         project_name=project_name
    #     )
    # tuner.search(x=[0], y=[1])

    # dir = r"Hyperparam_kt_A3C_LSTM"
    # project_name = "keras_tunning_soft"
    # tuner_lstm = kt.BayesianOptimization(
    #         MyHyperModel(hyper_dir = dir+"/"+project_name, writer = "logs_A3C_LSTM/A3C_soft/" ,use_LSTM = True),
    #         objective= kt.Objective('total_train_reward', direction="max"), 
    #         max_trials = 40,
    #         # distribution_strategy= strategy,
    #         directory=dir,
    #         project_name='keras_tunning'
    #     )
    # tuner_lstm.search(x=[0], y=[1])

    # project_name = "keras_tunning_epsilon"
    # tuner_lstm = kt.BayesianOptimization(
    #         MyHyperModel(hyper_dir = dir+"/"+project_name, writer = "logs_A3C_LSTM/A3C_epsilon/" ,use_LSTM = True, exploration_tech = "epsilon"),
    #         objective= kt.Objective('total_train_reward', direction="max"), 
    #         max_trials = 40,
    #         # distribution_strategy= strategy,
    #         directory=dir,
    #         project_name='keras_tunning'
    #     )
    # tuner_lstm.search(x=[0], y=[1])

    # project_name = "keras_tunning_boltzman"
    # tuner_lstm = kt.BayesianOptimization(
    #         MyHyperModel(hyper_dir = dir+"/"+project_name, writer = "logs_A3C_LSTM/A3C_boltzman/" ,use_LSTM = True, exploration_tech = "boltzman"),
    #         objective= kt.Objective('total_train_reward', direction="max"), 
    #         max_trials = 40,
    #         # distribution_strategy= strategy,
    #         directory=dir,
    #         project_name='keras_tunning'
    #     )
    # tuner_lstm.search(x=[0], y=[1])
   
else : 
    
    for exploration_tech in ['boltzman', 'epsilon']:
        print("Acquiring parameters ....")
        writer= "Training/fit_A3C/"

        n_enviroment = 10
        training_steps = 1000000
        learning_rate= 0.001
        entropy_factor = 0.1
        discount = 0.99
        dense_units = 512
        lstm_units = 128
        time_to_update= 100
        end_of_episode = 300
        use_LSTM = False

        
        run_training(training_steps, learning_rate, entropy_factor, exploration_tech, discount, time_to_update, dense_units, lstm_units, n_enviroment, writer, use_LSTM, end_of_episode, environment_name = "LunarLander-v2")
            

Trial 50 Complete [00h 22m 12s]
total_train_reward: -85.89

Best total_train_reward So Far: -80.96
Total elapsed time: 16h 08m 24s
INFO:tensorflow:Oracle triggered exit


In [11]:
exploration_tech = "soft"

hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=3):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

    n_env = 30
    end_ep = 1000
    ep = 1
    bolt_fact = 1
    train_steps = 5000000
    disc = trials.hyperparameters.values["discount"]
    lr = trials.hyperparameters.values["learning_rate"]
    entropy_fact = trials.hyperparameters.values["entropy_factor"]
    d = trials.hyperparameters.values["dense_units"]
    time_to_update = 100 if exploration_tech == "soft" else trials.hyperparameters.values["time_to_update"]
    ulstm = False

    hyperparam_combination.append((n_env, disc, end_ep, lr, entropy_fact, ep, bolt_fact, exploration_tech, train_steps, d, time_to_update, ulstm))

Trial id :11 | Score :-80.96 --> {'discount': 0.99, 'entropy_factor': 0.017541715374253038, 'learning_rate': 0.0017291290664505736, 'dense_units': 246}
Trial id :02 | Score :-81.31 --> {'discount': 0.99, 'entropy_factor': 0.0350890943274685, 'learning_rate': 0.0007764318837669324, 'dense_units': 293}
Trial id :04 | Score :-81.52 --> {'discount': 0.99, 'entropy_factor': 0.011796644340093837, 'learning_rate': 0.0020702575051384426, 'dense_units': 119}


In [12]:
len(hyperparam_combination), hyperparam_combination

(3,
 [(30,
   0.99,
   1000,
   0.0017291290664505736,
   0.017541715374253038,
   1,
   1,
   'soft',
   5000000,
   246,
   100,
   False),
  (30,
   0.99,
   1000,
   0.0007764318837669324,
   0.0350890943274685,
   1,
   1,
   'soft',
   5000000,
   293,
   100,
   False),
  (30,
   0.99,
   1000,
   0.0020702575051384426,
   0.011796644340093837,
   1,
   1,
   'soft',
   5000000,
   119,
   100,
   False)])

In [13]:
# run_hyperparam(TUNING_TYPE= "MANUAL", TUNING_TYPE = TUNING_TYPE, hyperparam_combination = hyperparam_combination,  total_files = 4)

In [14]:

val_env = gym.make(ENV, render_mode = "rgb_array")
dir = r"Hyperparam_kt_A3C"

for exploration_tech in ["soft" ,"epsilon", "boltzman"]:

    project_name = "keras_tunning_"+exploration_tech

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/A3C_"+exploration_tech+"/", exploration_tech =exploration_tech, 
                          end_of_episode = EPISODES, n_enviroment = 5, 
                  evaluation_epoch = 2000, training_steps = 600000,
                  sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                  discount_min = 0.90, discount_max = 0.99,
                  entropy_min = 0.0001, entropy_max = 0.05,
                  lr_min = 0.00001, lr_max = 0.005,
                  dense_min = 32, dense_max = 500,
                  lstm_min = 32, lstm_max = 128,
                  time_to_update_min = 100, time_to_update_max=600,
                  environment_name=ENV),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 50,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name
        )

    for trials in tuner.oracle.get_best_trials(num_trials=1):
        print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
        

        # env_model = tuner.get_best_models()[0]
        n_enviroment = 30
        end_ep = 1000
        ep = 1
        bolt_fact = 1
        training_steps = 5000000
        discount = trials.hyperparameters.values["discount"]
        learning_rate = trials.hyperparameters.values["learning_rate"]
        entropy_factor = trials.hyperparameters.values["entropy_factor"]
        dense_units = trials.hyperparameters.values["dense_units"]
        sigma_noise = trials.hyperparameters.values["sigma_noise"]
        time_to_update= 100
        lstm_units= 32
        use_LSTM=False
        end_of_episode = EPISODES
        save_factor=50000
        sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH
        sucess_criteria_value = SUCESS_CRITERIA_VALUE
        environment_name = ENV
        reward_scaler = 1
        continuous_space_actions = True
        return_agent = True

    env_model = run_training(training_steps, learning_rate, entropy_factor, exploration_tech, discount, time_to_update, dense_units, lstm_units, 
                    n_enviroment, writer, use_LSTM, end_of_episode, save_factor, sucess_criteria_epochs , sucess_criteria_value , 
                    environment_name, reward_scaler, continuous_space_actions, sigma_noise, return_agent)

    final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./A3C_"+exploration_tech+"_video.mp4")
    print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))

    

INFO:tensorflow:Reloading Tuner from Hyperparam_kt_A3C/keras_tunning_soft/tuner0.json
Trial id :11 | Score :-80.96 --> {'discount': 0.99, 'entropy_factor': 0.017541715374253038, 'learning_rate': 0.0017291290664505736, 'dense_units': 246}


KeyError: 'sigma_noise'